# Predicting Sex Assigned at Birth with Regression

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
from collections import OrderedDict
import torch
torch.manual_seed(0)
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
from abcd.local.paths import output_path
from abcd.data.read_data import get_subjects_events_sf, subject_cols_to_events
import abcd.data.VARS as VARS
from abcd.data.define_splits import SITES, save_restore_sex_fmri_splits
from abcd.data.divide_with_splits import divide_events_by_splits
from abcd.data.var_tailoring.normalization import normalize_var
from abcd.data.pytorch.get_dataset import PandasDataset

#regresssion-specific imports
from abcd.models.regression.MLPRegressor import MLPRegressor, LinearRegressor, MLPRegressorCustom
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from abcd.training.RegressorTrainer import RegressorTrainer

#plotting
import matplotlib.pyplot as plt
import pygal
from abcd.plotting.pygal.rendering import display_html
from sklearn.metrics import confusion_matrix
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from abcd.plotting.seaborn.confusion_matrix import plot_confusion_matrix

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
from datetime import datetime

In [4]:
from abcd.analysis.regression import preprocess, train_model

In [5]:
bucketing_scheme = "sex"

In [6]:
# Determine device for training (TODO: figure out why doesn't work with mps)
device = "cpu" #("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [7]:
dataloaders, events_train, events_id_test, events_ood_test, feature_cols = preprocess('kbi_sex_assigned_at_birth', ['fmri', 'smri'], ood_site_num=0)

There are 9632 subjects and 18808 visits with imaging
Leaving baseline visits, we have 9085 events

4703 visits with Male target
4382 visits with Female target

Created splits
Nr. events: 7063 train, 1738 val, 284 test

Created dataloaders
Shape and datatype of X: torch.Size([64, 737]), torch.float32
Shape and datatype of y: torch.Size([64]), torch.float32


# Linear Regression Baseline

In [8]:
#modify
config = {'target_col': 'kbi_sex_assigned_at_birth',
          'features': ['fmri', 'smri'],
          'model': ['abcd.models.regression.MLPRegressor', 'LinearRegressor'],
          'lr': 1e-3,
          'batch_size': 64,
          'nr_epochs': 1000
        }

#leave unmodified
experiment_title = 'ABCD_sex_' + config['model'][1] + "_" + datetime.now().strftime("%Y-%m-%d %H:%M:%S") #for saving results
models_path = os.path.join(output_path, experiment_title, 'models')

In [9]:
model = LinearRegressor(save_path=models_path, input_size=len(feature_cols)) #modfiy

model = model.to(device)
print(model)
trainer = train_model(model, device, config, experiment_title, dataloaders, verbose=True, bucketing_scheme=bucketing_scheme)
best_model_details = trainer.export_best_model(config=config)

LinearRegressor(
  (sigmoid): Sigmoid()
  (linear1): Linear(in_features=737, out_features=1, bias=True)
  (linear_layers): Linear(in_features=737, out_features=1, bias=True)
)


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 0
train MSELoss: 0.246 MSE: 0.246 MAE: 0.488 accuracy: 0.525
val MSELoss: 0.246 MSE: 0.245 MAE: 0.489 accuracy: 0.526
test MSELoss: 0.249 MSE: 0.249 MAE: 0.494 accuracy: 0.514

New best model.

Saved PyTorch model state LinearRegressor_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-25 11:51:11/models
Saved trainer state RegressorTrainer_optimizer_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-25 11:51:11/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-25 11:51:11/results


  0%|          | 1/1000 [00:00<04:13,  3.94it/s]

Ending epoch 1, loss 0.22392568609735988


...

Epoch 980
train MSELoss: 0.085 MSE: 0.085 MAE: 0.191 accuracy: 0.886
val MSELoss: 0.102 MSE: 0.101 MAE: 0.208 accuracy: 0.857
test MSELoss: 0.113 MSE: 0.111 MAE: 0.219 accuracy: 0.838
Ending epoch 981, loss 0.0841246849379024


 99%|█████████▉| 993/1000 [01:20<00:00, 12.29it/s]

Epoch 990
train MSELoss: 0.084 MSE: 0.084 MAE: 0.191 accuracy: 0.888
val MSELoss: 0.102 MSE: 0.100 MAE: 0.207 accuracy: 0.859
test MSELoss: 0.112 MSE: 0.110 MAE: 0.219 accuracy: 0.845
Ending epoch 991, loss 0.08391110062062203


100%|██████████| 1000/1000 [01:21<00:00, 12.28it/s]


Finished training
Epoch 1000
train MSELoss: 0.083 MSE: 0.083 MAE: 0.188 accuracy: 0.894
val MSELoss: 0.101 MSE: 0.099 MAE: 0.205 accuracy: 0.867
test MSELoss: 0.105 MSE: 0.103 MAE: 0.212 accuracy: 0.852
Saved PyTorch model state LinearRegressor_epoch1000.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-25 11:51:11/models
Saved trainer state RegressorTrainer_optimizer_epoch1000.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-25 11:51:11/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-25 11:51:11/results


# Custom MLP

In [8]:
#modify
experiment_title = 'sex_MLPReg_' + datetime.now().strftime("%Y-%m-%d %H:%M:%S") #for saving results
print(experiment_title)

config = {'target_col': 'kbi_sex_assigned_at_birth',
          'features': ['fmri', 'smri'],
          'model': ['abcd.models.regression.MLPRegressor', 'MLPRegressorCustom'],
          'batch_size': 64,

          #tune
          'lr': 1e-3,
          'nr_epochs': 150,
          'hidden_sizes': [256, 64]
        }

#leave unmodified
models_path = os.path.join(output_path, experiment_title, 'models')

sex_MLPReg_2023-08-25 06:58:52


In [9]:
model = MLPRegressorCustom(save_path=models_path, input_size=len(feature_cols), hidden_sizes=config['hidden_sizes']) #modfiy

model = model.to(device)
print(model)
trainer = train_model(model, device, config, experiment_title, dataloaders, verbose=True, bucketing_scheme=bucketing_scheme)
trainer.export_best_model(config=config)

MLPRegressorCustom(
  (sigmoid): Sigmoid()
  (linear_layers): Sequential(
    (0): Linear(in_features=737, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 0
train MSELoss: 0.252 MSE: 0.252 MAE: 0.491 accuracy: 0.517
val MSELoss: 0.252 MSE: 0.252 MAE: 0.491 accuracy: 0.520
test MSELoss: 0.254 MSE: 0.254 MAE: 0.493 accuracy: 0.507

New best model.

Saved PyTorch model state MLPRegressorCustom_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/sex_MLPReg_2023-08-25 06:58:52/models
Saved trainer state RegressorTrainer_optimizer_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/sex_MLPReg_2023-08-25 06:58:52/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/sex_MLPReg_2023-08-25 06:58:52/results


  1%|          | 1/150 [00:00<01:31,  1.62it/s]

Ending epoch 1, loss 0.21174884016986364


...

 94%|█████████▍| 141/150 [00:54<00:03,  2.35it/s]

Ending epoch 141, loss 0.09725392576273498


100%|██████████| 150/150 [00:57<00:00,  2.61it/s]


Finished training
Epoch 150
train MSELoss: 0.076 MSE: 0.076 MAE: 0.152 accuracy: 0.900
val MSELoss: 0.105 MSE: 0.102 MAE: 0.178 accuracy: 0.862
test MSELoss: 0.115 MSE: 0.112 MAE: 0.188 accuracy: 0.849
Saved PyTorch model state MLPRegressorCustom_epoch150.pth in /Users/carolinezanze/Desktop/abcd5_output/sex_MLPReg_2023-08-25 06:58:52/models
Saved trainer state RegressorTrainer_optimizer_epoch150.pth in /Users/carolinezanze/Desktop/abcd5_output/sex_MLPReg_2023-08-25 06:58:52/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/sex_MLPReg_2023-08-25 06:58:52/results


{'state_dict': OrderedDict([('linear_layers.0.weight',
               tensor([[ 0.0195, -0.0033, -0.1237,  ..., -0.0884,  0.0488, -0.0142],
                       [-0.0136,  0.0685, -0.0477,  ...,  0.0111, -0.0553,  0.0173],
                       [-0.0522,  0.0318, -0.0063,  ...,  0.0560, -0.0402, -0.0233],
                       ...,
                       [-0.0737, -0.0332, -0.0649,  ..., -0.0247,  0.0716, -0.0088],
                       [-0.0759, -0.0008, -0.0101,  ...,  0.0561,  0.0313, -0.0203],
                       [ 0.0296,  0.0517, -0.0352,  ..., -0.0312,  0.0461,  0.0606]])),
              ('linear_layers.0.bias',
               tensor([ 0.0222, -0.0067, -0.0284, -0.0056, -0.0425, -0.0450, -0.0043,  0.0000,
                        0.0000, -0.0056,  0.0014, -0.0086,  0.0410, -0.0059,  0.0000, -0.0082,
                       -0.0030,  0.0000,  0.0000,  0.0108,  0.0246, -0.0072, -0.0051, -0.0066,
                        0.0000, -0.0034,  0.0000, -0.0070, -0.0072, -0.0203, -0.

# Hyperparameter Search with Cusom MLP

In [8]:
hidden_sizes = [
    (128, 64),
    (256, 128, 64), 
    (512, 256, 128, 64),
]

learning_rates = [1e-3, 1e-5, 1e-7]

In [10]:
experiments = {}
global_best_val_mse = float('inf')
best_model_experiment_name = None
best_model = None

for i,learning_rate in enumerate(learning_rates):
    for j,sizes in enumerate(hidden_sizes):
        experiment_title = 'sex_MLPReg_' + datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        experiment_num = i*len(hidden_sizes) + j
        print("experiment", experiment_num, ":", experiment_title)

        config = {'target_col': 'kbi_sex_assigned_at_birth',
          'features': ['fmri', 'smri'],
          'model': ['abcd.models.regression.MLPRegressor', 'MLPRegressorCustom'],
          'batch_size': 64,
          'nr_epochs': 250,

          #tune
          'lr': 1e-3,
          'hidden_sizes': [256, 64]
        }

        config['hidden_sizes'] = sizes
        config['lr'] = learning_rate

        #define and train model
        models_path = os.path.join(output_path, experiment_title, 'models')
        model = MLPRegressorCustom(save_path=models_path, input_size=len(feature_cols), hidden_sizes=config['hidden_sizes']) #modfiy
        model = model.to(device)
        trainer = train_model(model, device, config, experiment_title, dataloaders, verbose=False, bucketing_scheme=bucketing_scheme)
        details = trainer.export_best_model(config=config)
        
        #update best model
        local_best_val_mse = details['metrics']['val']['MSE']
        if local_best_val_mse < global_best_val_mse:
            global_best_val_mse = local_best_val_mse
            best_model = details
            best_model_experiment_name = experiment_title

        #save experiment
        experiments[experiment_title] = details

print("\n\nExperiment over. Best model:", best_model_experiment_name)

experiment 0 : sex_MLPReg_2023-08-25 07:06:50


100%|██████████| 250/250 [01:13<00:00,  3.38it/s]


experiment 1 : sex_MLPReg_2023-08-25 07:08:05


100%|██████████| 250/250 [01:54<00:00,  2.19it/s]


experiment 2 : sex_MLPReg_2023-08-25 07:09:59


...

experiment 7 : sex_MLPReg_2023-08-25 07:20:40


100%|██████████| 250/250 [02:03<00:00,  2.03it/s]


experiment 8 : sex_MLPReg_2023-08-25 07:22:45


100%|██████████| 250/250 [03:24<00:00,  1.22it/s]




Experiment over. Best model: sex_MLPReg_2023-08-25 07:09:59 
 {'state_dict': OrderedDict([('linear_layers.0.weight', tensor([[ 0.0245,  0.0467, -0.0403,  ...,  0.0505, -0.0050,  0.0347],
        [-0.0262, -0.0278,  0.0033,  ..., -0.0476, -0.0188,  0.0350],
        [ 0.0027,  0.0385,  0.0554,  ...,  0.0008, -0.0375, -0.0542],
        ...,
        [-0.0637, -0.0446, -0.0767,  ..., -0.0479, -0.0501, -0.0743],
        [ 0.0635, -0.0462, -0.0167,  ...,  0.0128,  0.0647, -0.0221],
        [-0.1490, -0.1099, -0.0542,  ..., -0.0299,  0.0499,  0.0479]])), ('linear_layers.0.bias', tensor([-8.2581e-03, -5.0180e-03, -3.3046e-03, -5.5849e-03, -2.1489e-02,
         0.0000e+00,  0.0000e+00, -5.1597e-03, -4.2635e-03, -7.5176e-03,
         0.0000e+00,  2.3924e-03,  1.9830e-02,  0.0000e+00, -4.6496e-03,
        -6.0054e-03, -3.7849e-03, -6.5353e-03, -4.4883e-03,  0.0000e+00,
        -1.1326e-02, -1.2418e-02,  0.0000e+00, -3.0849e-02, -9.6788e-03,
        -6.8603e-03, -4.7983e-03, -8.4239e-03,  1.5392e